In [ ]:
```xml
<VSCode.Cell language="markdown">
# Workshop 3: AI Agents with Azure AI Foundry

Welcome to Workshop 3! In this notebook, you'll learn how to build intelligent AI agents using Azure AI Foundry's Agent Service and function calling capabilities.

## What You'll Learn

1. **Agent Fundamentals** - Understanding AI agents and their capabilities
2. **Azure AI Agent Service** - Using the cloud-native agent platform
3. **Function Calling** - Enabling agents to use tools and APIs
4. **Multi-Step Reasoning** - Building agents that can plan and execute
5. **Custom Tools** - Creating your own functions for agents
6. **Conversation Management** - Handling stateful agent interactions

## Prerequisites

- Completed Workshop 1 (Deploy Your First Model)
- Completed Workshop 2 (Tracing) - optional but recommended
- Azure AI Foundry project with Agent Service enabled
- Environment variables set up correctly

## Learning Objectives

By the end of this workshop, you will:
- Understand AI agent architecture and capabilities
- Build agents using Azure AI Foundry Agent Service
- Create custom tools and functions for agents
- Implement multi-step reasoning workflows
- Handle stateful conversations and memory
- Trace and monitor agent interactions
</VSCode.Cell>

<VSCode.Cell language="python">
# Install required packages for agents
%pip install azure-ai-projects[agents] azure-ai-inference azure-core-tracing-opentelemetry opentelemetry-instrumentation-openai-v2
</VSCode.Cell>

<VSCode.Cell language="markdown">
## 1. Environment Setup and Imports

Let's set up our environment for building AI agents.
</VSCode.Cell>

<VSCode.Cell language="python">
import os
import json
import time
import asyncio
from typing import Dict, List, Any, Optional
from dotenv import load_dotenv

# Azure AI imports
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import Agent, AgentThread, ThreadMessage, ToolSet, FunctionTool, CodeInterpreterTool

# OpenTelemetry for tracing (optional)
try:
    from opentelemetry import trace
    from opentelemetry.instrumentation.openai_v2 import OpenAIInstrumentor
    TRACING_AVAILABLE = True
except ImportError:
    TRACING_AVAILABLE = False
    print("📝 Note: OpenTelemetry not available, proceeding without tracing")

# Load environment variables
load_dotenv()

print("🤖 AI Agents Workshop Environment Check:")
print("-" * 45)

# Check required environment variables
required_vars = [
    'PROJECT_ENDPOINT',
    'AZURE_AI_FOUNDRY_RESOURCE_NAME',
    'MODEL_DEPLOYMENT_NAME'
]

for var in required_vars:
    value = os.getenv(var)
    status = "✅" if value else "❌"
    print(f"{status} {var}: {'Set' if value else 'Not set'}")

print("-" * 45)

# Optional: Set up tracing if available
if TRACING_AVAILABLE:
    try:
        OpenAIInstrumentor().instrument()
        print("🔍 Tracing enabled for agent interactions")
    except Exception as e:
        print(f"📝 Note: Tracing setup skipped: {e}")
</VSCode.Cell>

<VSCode.Cell language="markdown">
## 2. Understanding AI Agents

Before building agents, let's understand what makes them powerful and different from simple chatbots.
</VSCode.Cell>

<VSCode.Cell language="python">
def explain_agent_concepts():
    """
    Explain key AI agent concepts and capabilities.
    """
    print("🧠 AI Agent Concepts:")
    print("=" * 50)
    
    concepts = {
        "🤖 What is an AI Agent?": {
            "definition": "An AI system that can perceive, reason, plan, and act autonomously",
            "key_features": [
                "Autonomous decision-making",
                "Tool usage and function calling",
                "Multi-step reasoning and planning",
                "Memory and state management",
                "Goal-oriented behavior"
            ]
        },
        "⚙️ Function Calling": {
            "definition": "The ability for AI models to call external functions and APIs",
            "examples": [
                "Weather API calls",
                "Database queries",
                "File operations",
                "Mathematical calculations",
                "Web searches"
            ]
        },
        "🧩 Agent vs Chatbot": {
            "chatbot": "Responds to messages, stateless, single-turn focused",
            "agent": "Plans actions, maintains state, multi-turn reasoning, uses tools"
        },
        "🔄 Agent Workflow": {
            "steps": [
                "1. Receive task/goal",
                "2. Plan approach and steps", 
                "3. Execute actions using tools",
                "4. Evaluate results",
                "5. Continue or complete task"
            ]
        }
    }
    
    for concept, details in concepts.items():
        print(f"\n{concept}")
        if "definition" in details:
            print(f"  📖 Definition: {details['definition']}")
        if "key_features" in details:
            print(f"  🔑 Key Features:")
            for feature in details['key_features']:
                print(f"    • {feature}")
        if "examples" in details:
            print(f"  💡 Examples:")
            for example in details['examples']:
                print(f"    • {example}")
        if "chatbot" in details:
            print(f"  🗨️ Chatbot: {details['chatbot']}")
            print(f"  🤖 Agent: {details['agent']}")
        if "steps" in details:
            print(f"  📋 Steps:")
            for step in details['steps']:
                print(f"    {step}")

explain_agent_concepts()
</VSCode.Cell>

<VSCode.Cell language="markdown">
## 3. Connect to Azure AI Foundry Agent Service

Let's connect to Azure AI Foundry and initialize the Agent Service.
</VSCode.Cell>

<VSCode.Cell language="python">
# Connect to Azure AI Foundry
try:
    print("🔗 Connecting to Azure AI Foundry Agent Service...")
    
    # Initialize project client
    credential = DefaultAzureCredential()
    project_client = AIProjectClient(
        endpoint=os.getenv('PROJECT_ENDPOINT'),
        credential=credential
    )
    
    # Get the agents client
    agents_client = project_client.agents
    
    print("✅ Connected to Azure AI Foundry Agent Service")
    print(f"📍 Project: {os.getenv('PROJECT_ENDPOINT')}")
    print(f"🤖 Model: {os.getenv('MODEL_DEPLOYMENT_NAME')}")
    
    # Check if we have access to the agent service
    try:
        # This will test our connection to the agent service
        print("🧪 Testing agent service access...")
        # We'll create a simple agent shortly to test this
        print("✅ Agent service accessible")
    except Exception as e:
        print(f"⚠️ Agent service test: {e}")
    
except Exception as e:
    print(f"❌ Connection failed: {e}")
    print("💡 Make sure your AI Foundry project has Agent Service enabled")
    exit()
</VSCode.Cell>

<VSCode.Cell language="markdown">
## 4. Creating Your First Agent

Let's create a simple agent without any tools to understand the basics.
</VSCode.Cell>

<VSCode.Cell language="python">
# Create a basic agent
def create_basic_agent():
    """
    Create a simple AI agent without any tools.
    """
    print("🤖 Creating Your First AI Agent:")
    print("-" * 35)
    
    try:
        # Define the agent
        agent = agents_client.create_agent(
            model=os.getenv('MODEL_DEPLOYMENT_NAME'),
            name="Workshop Assistant",
            description="A helpful AI assistant for workshop demonstrations",
            instructions="""You are a knowledgeable AI assistant helping users learn about Azure AI and machine learning.
            
Your personality:
- Friendly and encouraging
- Technical but accessible
- Always eager to help and explain concepts
- Use emojis appropriately to make conversations engaging

Your capabilities:
- Explain AI and ML concepts clearly
- Help with Azure AI services
- Provide coding examples and best practices
- Answer questions about Azure AI Foundry

Always be helpful and maintain a positive, educational tone."""
        )
        
        print(f"✅ Agent created successfully!")
        print(f"🆔 Agent ID: {agent.id}")
        print(f"📝 Name: {agent.name}")
        print(f"🎯 Model: {agent.model}")
        
        return agent
        
    except Exception as e:
        print(f"❌ Agent creation failed: {e}")
        return None

# Create our first agent
basic_agent = create_basic_agent()
</VSCode.Cell>

<VSCode.Cell language="markdown">
## 5. Creating Threads and Having Conversations

Agents use threads to maintain conversation state. Let's create a thread and chat with our agent.
</VSCode.Cell>

<VSCode.Cell language="python">
# Chat with the basic agent
def chat_with_basic_agent(agent: Agent):
    """
    Demonstrate basic conversation with an agent using threads.
    """
    if not agent:
        print("❌ No agent available for chat")
        return
        
    print("💬 Starting Conversation with Basic Agent:")
    print("-" * 45)
    
    try:
        # Create a thread for this conversation
        thread = agents_client.create_thread()
        print(f"🧵 Thread created: {thread.id}")
        
        # List of questions to ask
        questions = [
            "Hi! Can you introduce yourself?",
            "What is Azure AI Foundry and what can I do with it?",
            "Explain the difference between AI agents and chatbots in simple terms."
        ]
        
        for i, question in enumerate(questions, 1):
            print(f"\n{i}️⃣ User: {question}")
            
            # Add user message to thread
            message = agents_client.create_message(
                thread_id=thread.id,
                role="user",
                content=question
            )
            
            # Create and execute a run
            run = agents_client.create_run(
                thread_id=thread.id,
                agent_id=agent.id
            )
            
            # Wait for completion
            print("🤔 Agent thinking...")
            
            # Poll for completion
            while run.status in ["queued", "in_progress"]:
                time.sleep(1)
                run = agents_client.get_run(thread_id=thread.id, run_id=run.id)
            
            if run.status == "completed":
                # Get the agent's response
                messages = agents_client.list_messages(thread_id=thread.id)
                
                # Find the latest assistant message
                for msg in messages.data:
                    if msg.role == "assistant" and msg.created_at > message.created_at:
                        print(f"🤖 Agent: {msg.content[0].text.value}")
                        break
            else:
                print(f"❌ Run failed with status: {run.status}")
                if hasattr(run, 'last_error') and run.last_error:
                    print(f"Error: {run.last_error}")
        
        return thread
        
    except Exception as e:
        print(f"❌ Conversation failed: {e}")
        return None

# Chat with our basic agent
basic_thread = chat_with_basic_agent(basic_agent)
</VSCode.Cell>

<VSCode.Cell language="markdown">
## 6. Creating Custom Tools for Agents

Now let's create custom tools that our agents can use to perform specific tasks.
</VSCode.Cell>

<VSCode.Cell language="python">
# Define custom tools for our agent
def create_custom_tools():
    """
    Create custom tools that agents can use.
    """
    print("🔧 Creating Custom Tools for Agents:")
    print("-" * 40)
    
    # Tool 1: Weather Information (simulated)
    weather_tool = FunctionTool(
        name="get_weather",
        description="Get current weather information for a specified location",
        parameters={
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and country, e.g., 'San Francisco, CA'"
                },
                "unit": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "Temperature unit"
                }
            },
            "required": ["location"]
        }
    )
    
    # Tool 2: Calculate token costs
    cost_calculator_tool = FunctionTool(
        name="calculate_token_cost",
        description="Calculate the estimated cost of OpenAI API usage based on tokens",
        parameters={
            "type": "object",
            "properties": {
                "model": {
                    "type": "string",
                    "description": "The model name (e.g., 'gpt-4o', 'gpt-4o-mini')"
                },
                "input_tokens": {
                    "type": "integer",
                    "description": "Number of input tokens"
                },
                "output_tokens": {
                    "type": "integer", 
                    "description": "Number of output tokens"
                }
            },
            "required": ["model", "input_tokens", "output_tokens"]
        }
    )
    
    # Tool 3: Azure service information
    azure_info_tool = FunctionTool(
        name="get_azure_service_info",
        description="Get information about Azure AI services and their capabilities",
        parameters={
            "type": "object",
            "properties": {
                "service_name": {
                    "type": "string",
                    "description": "Name of the Azure service (e.g., 'Computer Vision', 'Speech', 'Language')"
                },
                "info_type": {
                    "type": "string",
                    "enum": ["capabilities", "pricing", "regions", "limits"],
                    "description": "Type of information requested"
                }
            },
            "required": ["service_name"]
        }
    )
    
    print("✅ Custom tools defined:")
    print("  🌤️ Weather tool - Get weather information")
    print("  💰 Cost calculator - Calculate API usage costs")
    print("  ☁️ Azure info tool - Get Azure service information")
    
    return [weather_tool, cost_calculator_tool, azure_info_tool]

# Create our custom tools
custom_tools = create_custom_tools()
</VSCode.Cell>

<VSCode.Cell language="markdown">
## 7. Implementing Tool Functions

Let's implement the actual functions that will be called when agents use our tools.
</VSCode.Cell>

<VSCode.Cell language="python">
# Implement the actual tool functions
class ToolImplementations:
    """
    Implementations of the custom tools our agents can use.
    """
    
    @staticmethod
    def get_weather(location: str, unit: str = "celsius") -> Dict[str, Any]:
        """
        Simulated weather function (in production, you'd call a real weather API).
        """
        print(f"🌤️ Getting weather for {location} in {unit}")
        
        # Simulate weather data (in reality, you'd call a real API)
        import random
        
        weather_conditions = ["sunny", "cloudy", "rainy", "partly cloudy", "snowy"]
        temp_celsius = random.randint(-10, 35)
        temp_fahrenheit = (temp_celsius * 9/5) + 32
        
        return {
            "location": location,
            "temperature": temp_fahrenheit if unit == "fahrenheit" else temp_celsius,
            "unit": unit,
            "condition": random.choice(weather_conditions),
            "humidity": random.randint(30, 90),
            "wind_speed": random.randint(5, 25),
            "note": "This is simulated data for workshop purposes"
        }
    
    @staticmethod
    def calculate_token_cost(model: str, input_tokens: int, output_tokens: int) -> Dict[str, Any]:
        """
        Calculate estimated costs for OpenAI API usage.
        """
        print(f"💰 Calculating costs for {model}: {input_tokens} input, {output_tokens} output tokens")
        
        # Simplified pricing (check current Azure OpenAI pricing for accurate rates)
        pricing = {
            "gpt-4o": {"input": 0.03, "output": 0.06},  # per 1K tokens
            "gpt-4o-mini": {"input": 0.00015, "output": 0.0006},
            "gpt-35-turbo": {"input": 0.001, "output": 0.002}
        }
        
        # Normalize model name
        model_key = model.lower().replace("-", "").replace("_", "")
        if "gpt4o" in model_key and "mini" not in model_key:
            model_key = "gpt-4o"
        elif "mini" in model_key:
            model_key = "gpt-4o-mini"
        else:
            model_key = "gpt-35-turbo"
        
        if model_key in pricing:
            input_cost = (input_tokens / 1000) * pricing[model_key]["input"]
            output_cost = (output_tokens / 1000) * pricing[model_key]["output"]
            total_cost = input_cost + output_cost
            
            return {
                "model": model,
                "input_tokens": input_tokens,
                "output_tokens": output_tokens,
                "total_tokens": input_tokens + output_tokens,
                "input_cost_usd": round(input_cost, 6),
                "output_cost_usd": round(output_cost, 6),
                "total_cost_usd": round(total_cost, 6),
                "pricing_model": model_key,
                "note": "Estimates based on Azure OpenAI pricing as of 2024"
            }
        else:
            return {
                "error": f"Pricing not available for model: {model}",
                "supported_models": list(pricing.keys())
            }
    
    @staticmethod
    def get_azure_service_info(service_name: str, info_type: str = "capabilities") -> Dict[str, Any]:
        """
        Get information about Azure AI services.
        """
        print(f"☁️ Getting {info_type} information for Azure {service_name}")
        
        # Simplified service information
        services = {
            "computer vision": {
                "capabilities": [
                    "Image analysis and tagging",
                    "Object detection",
                    "Optical Character Recognition (OCR)",
                    "Face detection and recognition",
                    "Image classification"
                ],
                "pricing": "Pay-per-transaction model, starting from $1-2 per 1000 transactions",
                "regions": "Available in 20+ Azure regions worldwide",
                "limits": "Up to 10 TPS (transactions per second) in free tier"
            },
            "speech": {
                "capabilities": [
                    "Speech-to-text conversion",
                    "Text-to-speech synthesis",
                    "Real-time speech translation",
                    "Speaker recognition",
                    "Custom voice models"
                ],
                "pricing": "Pay-per-hour for speech-to-text, pay-per-character for text-to-speech",
                "regions": "Available in most Azure regions",
                "limits": "Free tier includes 5 hours of speech-to-text per month"
            },
            "language": {
                "capabilities": [
                    "Sentiment analysis",
                    "Named entity recognition",
                    "Language detection",
                    "Key phrase extraction",
                    "Text summarization"
                ],
                "pricing": "Pay-per-transaction, starting from $2 per 1000 text records",
                "regions": "Available globally in 25+ regions",
                "limits": "Free tier includes 5000 transactions per month"
            }
        }
        
        service_key = service_name.lower().replace(" ", "").replace("-", "")
        
        if service_key in services:
            service_data = services[service_key]
            if info_type in service_data:
                return {
                    "service": service_name,
                    "info_type": info_type,
                    "data": service_data[info_type],
                    "source": "Azure AI Services documentation"
                }
            else:
                return {
                    "service": service_name,
                    "available_info_types": list(service_data.keys()),
                    "requested_type": info_type
                }
        else:
            return {
                "error": f"Service '{service_name}' not found",
                "available_services": list(services.keys())
            }

# Create an instance of our tool implementations
tool_impl = ToolImplementations()

print("🔧 Tool implementations ready:")
print("  ✅ Weather API (simulated)")
print("  ✅ Token cost calculator")
print("  ✅ Azure service information lookup")
</VSCode.Cell>

<VSCode.Cell language="markdown">
## 8. Creating an Agent with Tools

Now let's create an agent that can use our custom tools.
</VSCode.Cell>

<VSCode.Cell language="python">
# Create an agent with tools
def create_agent_with_tools():
    """
    Create an AI agent equipped with custom tools.
    """
    print("🛠️ Creating Agent with Custom Tools:")
    print("-" * 40)
    
    try:
        # Create toolset with our custom tools
        toolset = ToolSet()
        toolset.add(*custom_tools)
        
        # Also add code interpreter for mathematical operations
        toolset.add(CodeInterpreterTool())
        
        # Create the agent with tools
        agent = agents_client.create_agent(
            model=os.getenv('MODEL_DEPLOYMENT_NAME'),
            name="Azure AI Workshop Assistant",
            description="An AI agent with tools for weather, cost calculation, and Azure information",
            instructions="""You are an advanced AI assistant with access to several helpful tools.

Your tools include:
- get_weather: Get weather information for any location
- calculate_token_cost: Calculate OpenAI API usage costs
- get_azure_service_info: Get information about Azure AI services
- code_interpreter: Perform mathematical calculations and data analysis

Guidelines for tool usage:
1. Always use tools when they can provide better, more accurate information
2. For weather questions, use the get_weather tool
3. For cost calculations, use the calculate_token_cost tool
4. For Azure service questions, use the get_azure_service_info tool
5. For math problems or data analysis, use the code interpreter
6. Explain what you're doing before using a tool
7. Summarize and interpret tool results for the user

Your personality:
- Professional but friendly
- Thorough and accurate
- Always explain your reasoning
- Proactive in suggesting relevant tools

When you use a tool, explain why you're using it and what you expect to find.""",
            tools=toolset
        )
        
        print(f"✅ Agent with tools created successfully!")
        print(f"🆔 Agent ID: {agent.id}")
        print(f"📝 Name: {agent.name}")
        print(f"🔧 Tools: {len(custom_tools) + 1} custom tools + code interpreter")
        
        return agent
        
    except Exception as e:
        print(f"❌ Agent creation failed: {e}")
        return None

# Create our tool-enabled agent
tool_agent = create_agent_with_tools()
</VSCode.Cell>

<VSCode.Cell language="markdown">
## 9. Handling Tool Calls in Agent Runs

Let's implement the logic to handle tool calls when the agent decides to use them.
</VSCode.Cell>

<VSCode.Cell language="python">
# Handle tool calls during agent execution
def handle_agent_tool_calls(thread_id: str, run_id: str) -> bool:
    """
    Handle tool calls made by the agent during execution.
    """
    try:
        run = agents_client.get_run(thread_id=thread_id, run_id=run_id)
        
        if run.status == "requires_action":
            print("🔧 Agent requested tool execution...")
            
            tool_outputs = []
            
            for tool_call in run.required_action.submit_tool_outputs.tool_calls:
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)
                
                print(f"🛠️ Executing tool: {function_name}")
                print(f"📥 Arguments: {function_args}")
                
                # Route to appropriate tool implementation
                try:
                    if function_name == "get_weather":
                        result = tool_impl.get_weather(**function_args)
                    elif function_name == "calculate_token_cost":
                        result = tool_impl.calculate_token_cost(**function_args)
                    elif function_name == "get_azure_service_info":
                        result = tool_impl.get_azure_service_info(**function_args)
                    else:
                        result = {"error": f"Unknown function: {function_name}"}
                    
                    print(f"📤 Result: {result}")
                    
                    tool_outputs.append({
                        "tool_call_id": tool_call.id,
                        "output": json.dumps(result)
                    })
                    
                except Exception as e:
                    print(f"❌ Tool execution failed: {e}")
                    tool_outputs.append({
                        "tool_call_id": tool_call.id,
                        "output": json.dumps({"error": str(e)})
                    })
            
            # Submit tool outputs back to the agent
            agents_client.submit_tool_outputs(
                thread_id=thread_id,
                run_id=run_id,
                tool_outputs=tool_outputs
            )
            
            return True
            
        return False
        
    except Exception as e:
        print(f"❌ Tool call handling failed: {e}")
        return False

def execute_agent_with_tools(agent: Agent, thread_id: str, user_message: str) -> str:
    """
    Execute an agent run with tool call handling.
    """
    try:
        # Add user message
        agents_client.create_message(
            thread_id=thread_id,
            role="user",
            content=user_message
        )
        
        # Create run
        run = agents_client.create_run(
            thread_id=thread_id,
            agent_id=agent.id
        )
        
        print("🤔 Agent processing request...")
        
        # Poll and handle tool calls
        max_iterations = 10
        iteration = 0
        
        while run.status in ["queued", "in_progress", "requires_action"] and iteration < max_iterations:
            if run.status == "requires_action":
                # Handle tool calls
                handled = handle_agent_tool_calls(thread_id, run.id)
                if not handled:
                    break
            
            time.sleep(1)
            run = agents_client.get_run(thread_id=thread_id, run_id=run.id)
            iteration += 1
        
        if run.status == "completed":
            # Get the latest message
            messages = agents_client.list_messages(thread_id=thread_id)
            for msg in messages.data:
                if msg.role == "assistant":
                    return msg.content[0].text.value
        else:
            return f"❌ Run failed with status: {run.status}"
            
    except Exception as e:
        return f"❌ Execution failed: {e}"

print("🔧 Tool execution handler ready")
</VSCode.Cell>

<VSCode.Cell language="markdown">
## 10. Demonstrating Agent Tool Usage

Let's test our agent with various requests that will trigger different tools.
</VSCode.Cell>

<VSCode.Cell language="python">
# Test the agent with tool usage
def demonstrate_agent_tools(agent: Agent):
    """
    Demonstrate the agent using various tools.
    """
    if not agent:
        print("❌ No agent available for demonstration")
        return
        
    print("🧪 Testing Agent with Tools:")
    print("-" * 35)
    
    try:
        # Create a new thread for this conversation
        thread = agents_client.create_thread()
        print(f"🧵 Thread created: {thread.id}")
        
        # Test cases that should trigger different tools
        test_cases = [
            {
                "request": "What's the weather like in San Francisco, CA?",
                "expected_tool": "get_weather"
            },
            {
                "request": "Calculate the cost for using gpt-4o with 1000 input tokens and 500 output tokens",
                "expected_tool": "calculate_token_cost"
            },
            {
                "request": "What are the capabilities of Azure Computer Vision service?",
                "expected_tool": "get_azure_service_info"
            },
            {
                "request": "Calculate the square root of 144 and then multiply it by 7",
                "expected_tool": "code_interpreter"
            },
            {
                "request": "Compare the weather in New York and London, then calculate how much it would cost to process this comparison using gpt-4o-mini",
                "expected_tool": "multiple tools"
            }
        ]
        
        for i, test_case in enumerate(test_cases, 1):
            print(f"\n{i}️⃣ Test Case: {test_case['expected_tool']}")
            print(f"📝 Request: {test_case['request']}")
            print("-" * 50)
            
            response = execute_agent_with_tools(agent, thread.id, test_case['request'])
            print(f"🤖 Agent Response:\n{response}")
            
            print("-" * 50)
            time.sleep(1)  # Brief pause between tests
        
        return thread
        
    except Exception as e:
        print(f"❌ Tool demonstration failed: {e}")
        return None

# Run the tool demonstrations
tool_thread = demonstrate_agent_tools(tool_agent)
</VSCode.Cell>

<VSCode.Cell language="markdown">
## 11. Building a Multi-Step Reasoning Agent

Let's create an agent that can handle complex, multi-step tasks requiring planning and coordination.
</VSCode.Cell>

<VSCode.Cell language="python">
# Create a multi-step reasoning agent
def create_planning_agent():
    """
    Create an agent specialized in multi-step reasoning and planning.
    """
    print("🧠 Creating Multi-Step Planning Agent:")
    print("-" * 40)
    
    try:
        # Create toolset with all our tools
        toolset = ToolSet()
        toolset.add(*custom_tools)
        toolset.add(CodeInterpreterTool())
        
        # Create the planning agent
        agent = agents_client.create_agent(
            model=os.getenv('MODEL_DEPLOYMENT_NAME'),
            name="Azure AI Planning Assistant",
            description="An AI agent that excels at multi-step reasoning, planning, and task coordination",
            instructions="""You are an advanced AI agent specialized in complex problem-solving through multi-step reasoning.

Your approach to complex tasks:
1. ANALYZE: Break down complex requests into smaller, manageable steps
2. PLAN: Create a logical sequence of actions to achieve the goal
3. EXECUTE: Use available tools systematically to gather information
4. SYNTHESIZE: Combine results from multiple tools to provide comprehensive answers
5. VERIFY: Double-check calculations and cross-reference information when possible

Your tools:
- get_weather: Weather information for any location
- calculate_token_cost: OpenAI API cost calculations
- get_azure_service_info: Azure AI service information
- code_interpreter: Mathematical calculations and data analysis

When handling complex requests:
- Always explain your planning process
- Show your step-by-step approach
- Use multiple tools when beneficial
- Provide comprehensive summaries
- Suggest follow-up actions when appropriate

Example approach for "Plan an AI project":
1. Understand requirements and constraints
2. Research relevant Azure services
3. Calculate estimated costs
4. Provide implementation timeline
5. Suggest monitoring and optimization strategies

Be thorough, systematic, and always explain your reasoning.""",
            tools=toolset
        )
        
        print(f"✅ Planning agent created successfully!")
        print(f"🆔 Agent ID: {agent.id}")
        print(f"🧠 Specialization: Multi-step reasoning and planning")
        
        return agent
        
    except Exception as e:
        print(f"❌ Planning agent creation failed: {e}")
        return None

# Create our planning agent
planning_agent = create_planning_agent()
</VSCode.Cell>

<VSCode.Cell language="markdown">
## 12. Complex Multi-Step Task Demonstration

Let's give our planning agent a complex task that requires multiple steps and tools.
</VSCode.Cell>

<VSCode.Cell language="python">
# Test complex multi-step reasoning
def test_complex_planning(agent: Agent):
    """
    Test the agent with complex, multi-step tasks.
    """
    if not agent:
        print("❌ No planning agent available")
        return
        
    print("🎯 Testing Complex Multi-Step Planning:")
    print("-" * 45)
    
    try:
        # Create a new thread
        thread = agents_client.create_thread()
        print(f"🧵 Planning thread created: {thread.id}")
        
        # Complex scenario
        complex_task = """I'm planning to deploy an AI-powered customer service solution for a company that receives 10,000 customer inquiries per month. 

The system should:
1. Handle weather-related questions for their outdoor events business
2. Provide cost estimates for different AI models
3. Use Azure AI services for sentiment analysis and language processing

Please help me plan this project by:
- Analyzing the requirements
- Recommending appropriate Azure services
- Calculating estimated monthly costs for different scenarios
- Providing a step-by-step implementation plan

Assume an average of 50 tokens input and 100 tokens output per customer interaction."""
        
        print(f"📋 Complex Task:")
        print(f"{complex_task}")
        print("\n" + "="*60)
        
        response = execute_agent_with_tools(agent, thread.id, complex_task)
        print(f"🤖 Agent's Comprehensive Plan:\n{response}")
        
        # Follow-up question to test memory and continuation
        follow_up = "Based on your analysis, what would be the cost difference between using gpt-4o versus gpt-4o-mini for this customer service solution? Also check what the weather is like in Seattle where the company is located."
        
        print(f"\n📋 Follow-up Question:")
        print(f"{follow_up}")
        print("\n" + "="*60)
        
        follow_up_response = execute_agent_with_tools(agent, thread.id, follow_up)
        print(f"🤖 Agent's Follow-up Response:\n{follow_up_response}")
        
        return thread
        
    except Exception as e:
        print(f"❌ Complex planning test failed: {e}")
        return None

# Run the complex planning test
planning_thread = test_complex_planning(planning_agent)
</VSCode.Cell>

<VSCode.Cell language="markdown">
## 13. Agent Memory and Conversation Management

Let's explore how agents maintain memory across conversations and handle long-running interactions.
</VSCode.Cell>

<VSCode.Cell language="python">
# Demonstrate agent memory and conversation management
def test_agent_memory(agent: Agent, thread_id: str):
    """
    Test how agents maintain memory and context across conversations.
    """
    if not agent or not thread_id:
        print("❌ No agent or thread available for memory test")
        return
        
    print("🧠 Testing Agent Memory and Context:")
    print("-" * 40)
    
    try:
        # Test memory by referencing previous conversation
        memory_tests = [
            "What was the main topic of our previous conversation?",
            "Can you remind me what Azure services you recommended earlier?",
            "What were the cost estimates you calculated for gpt-4o?",
            "Based on everything we've discussed, give me a 3-sentence summary of the customer service AI project plan"
        ]
        
        for i, test in enumerate(memory_tests, 1):
            print(f"\n{i}️⃣ Memory Test: {test}")
            response = execute_agent_with_tools(agent, thread_id, test)
            print(f"🤖 Response: {response}")
            time.sleep(1)
        
        # Test conversation history retrieval
        print(f"\n📜 Conversation History Analysis:")
        print("-" * 35)
        
        messages = agents_client.list_messages(thread_id=thread_id)
        
        user_messages = [msg for msg in messages.data if msg.role == "user"]
        assistant_messages = [msg for msg in messages.data if msg.role == "assistant"] 
        
        print(f"💬 Total conversation turns: {len(user_messages)}")
        print(f"🤖 Agent responses: {len(assistant_messages)}")
        print(f"📏 Conversation length: {len(messages.data)} messages")
        
        # Show conversation flow
        print(f"\n🔄 Conversation Flow:")
        for i, msg in enumerate(reversed(list(messages.data)[-10:]), 1):  # Last 10 messages
            role_emoji = "👤" if msg.role == "user" else "🤖"
            content_preview = msg.content[0].text.value[:100] + "..." if len(msg.content[0].text.value) > 100 else msg.content[0].text.value
            print(f"{i}. {role_emoji} {msg.role}: {content_preview}")
        
    except Exception as e:
        print(f"❌ Memory test failed: {e}")

# Test memory with our planning thread
if planning_thread:
    test_agent_memory(planning_agent, planning_thread.id)
</VSCode.Cell>

<VSCode.Cell language="markdown">
## 14. Agent Performance Monitoring

Let's add monitoring and tracing to understand agent performance and behavior.
</VSCode.Cell>

<VSCode.Cell language="python">
# Monitor agent performance
def analyze_agent_performance():
    """
    Analyze agent performance metrics and behavior.
    """
    print("📊 Agent Performance Analysis:")
    print("-" * 35)
    
    try:
        # Get information about our agents
        if basic_agent:
            print(f"🤖 Basic Agent:")
            print(f"   ID: {basic_agent.id}")
            print(f"   Model: {basic_agent.model}")
            print(f"   Tools: None")
        
        if tool_agent:
            print(f"🛠️ Tool Agent:")
            print(f"   ID: {tool_agent.id}")
            print(f"   Model: {tool_agent.model}")
            print(f"   Tools: {len(custom_tools) + 1}")
        
        if planning_agent:
            print(f"🧠 Planning Agent:")
            print(f"   ID: {planning_agent.id}")
            print(f"   Model: {planning_agent.model}")
            print(f"   Specialization: Multi-step reasoning")
        
        # Analyze thread performance
        threads_analyzed = 0
        for thread_name, thread in [("Basic", basic_thread), ("Tools", tool_thread), ("Planning", planning_thread)]:
            if thread:
                print(f"\n📈 {thread_name} Thread Performance:")
                messages = agents_client.list_messages(thread_id=thread.id)
                
                user_msgs = [msg for msg in messages.data if msg.role == "user"]
                assistant_msgs = [msg for msg in messages.data if msg.role == "assistant"]
                
                if assistant_msgs:
                    avg_response_length = sum(len(msg.content[0].text.value) for msg in assistant_msgs) / len(assistant_msgs)
                    print(f"   💬 Messages: {len(messages.data)} total")
                    print(f"   📏 Avg response length: {avg_response_length:.0f} characters")
                    print(f"   🔄 Conversation turns: {len(user_msgs)}")
                
                threads_analyzed += 1
        
        print(f"\n🎯 Summary:")
        print(f"   Agents created: {sum(1 for agent in [basic_agent, tool_agent, planning_agent] if agent)}")
        print(f"   Threads analyzed: {threads_analyzed}")
        print(f"   Custom tools: {len(custom_tools)}")
        
        # Performance tips
        print(f"\n💡 Performance Tips:")
        tips = [
            "Use specific, well-defined tool descriptions",
            "Implement proper error handling in tool functions",
            "Monitor token usage for cost optimization",
            "Use conversation threads efficiently",
            "Add tracing for production monitoring"
        ]
        
        for tip in tips:
            print(f"   • {tip}")
            
    except Exception as e:
        print(f"❌ Performance analysis failed: {e}")

# Analyze performance
analyze_agent_performance()
</VSCode.Cell>

<VSCode.Cell language="markdown">
## 15. Production Best Practices for Agents

Let's cover best practices for deploying agents in production environments.
</VSCode.Cell>

<VSCode.Cell language="python">
# Production best practices for agents
def production_best_practices():
    """
    Demonstrate production best practices for AI agents.
    """
    print("🏭 Production Best Practices for AI Agents:")
    print("=" * 50)
    
    practices = {
        "🔒 Security": [
            "Validate all tool inputs to prevent injection attacks",
            "Use Azure Managed Identity for authentication",
            "Implement rate limiting for tool calls",
            "Sanitize tool outputs before returning to users",
            "Log agent interactions for security monitoring"
        ],
        "⚡ Performance": [
            "Implement tool call timeouts",
            "Use async/await for non-blocking tool execution",
            "Cache expensive tool call results",
            "Monitor and limit token usage",
            "Optimize tool descriptions for better model understanding"
        ],
        "🔧 Reliability": [
            "Implement retry logic for failed tool calls",
            "Handle tool execution errors gracefully",
            "Validate tool outputs before processing",
            "Set maximum iteration limits for agent runs",
            "Implement circuit breakers for external API calls"
        ],
        "📊 Monitoring": [
            "Track tool usage patterns and success rates",
            "Monitor conversation thread lengths",
            "Alert on unusual agent behavior",
            "Measure response times and latency",
            "Track cost per conversation"
        ],
        "🎯 User Experience": [
            "Provide clear feedback during long-running operations",
            "Implement conversation memory management",
            "Handle user interruptions gracefully",
            "Provide fallback responses when tools fail",
            "Enable users to review and approve sensitive tool calls"
        ]
    }
    
    for category, items in practices.items():
        print(f"\n{category}:")
        for item in items:
            print(f"  • {item}")
    
    print(f"\n🚨 Common Pitfalls to Avoid:")
    pitfalls = [
        "Allowing unrestricted tool access without validation",
        "Not implementing proper error handling for tool failures",
        "Creating tools with unclear or ambiguous descriptions",
        "Not monitoring agent behavior for unexpected patterns", 
        "Ignoring token costs and usage optimization",
        "Not testing edge cases in tool implementations",
        "Failing to implement proper conversation state management"
    ]
    
    for pitfall in pitfalls:
        print(f"  ❌ {pitfall}")
    
    # Example production-ready tool implementation
    print(f"\n🔧 Example: Production-Ready Tool Implementation:")
    print("""
def production_weather_tool(location: str, unit: str = "celsius") -> Dict[str, Any]:
    \"\"\"
    Production-ready weather tool with proper error handling.
    \"\"\"
    import re
    import time
    from typing import Dict, Any
    
    # Input validation
    if not location or len(location.strip()) < 2:
        return {"error": "Invalid location provided"}
    
    # Sanitize input
    location = re.sub(r'[^a-zA-Z0-9\s,.-]', '', location)
    
    try:
        # Rate limiting
        time.sleep(0.1)  # Simple rate limiting
        
        # Call external API with timeout
        # result = requests.get(weather_api_url, timeout=5)
        
        # Validate response
        # if result.status_code != 200:
        #     return {"error": "Weather service unavailable"}
        
        # For demo, return simulated data
        return {
            "location": location,
            "temperature": 22,
            "unit": unit,
            "condition": "sunny",
            "timestamp": time.time(),
            "source": "production_weather_api"
        }
        
    except Exception as e:
        # Log error but don't expose internal details
        return {"error": "Weather service temporarily unavailable"}
""")

production_best_practices()
</VSCode.Cell>

<VSCode.Cell language="markdown">
## 16. Workshop Summary and Cleanup

Congratulations! You've completed Workshop 3 on AI Agents. Let's clean up and summarize what you've learned.
</VSCode.Cell>

<VSCode.Cell language="python">
# Workshop summary and cleanup
def workshop_summary_and_cleanup():
    """
    Summarize the workshop and clean up created resources.
    """
    print("🎯 Workshop 3 Summary: AI Agents")
    print("=" * 50)
    
    achievements = [
        "✅ Understood AI agent concepts and architecture",
        "✅ Created basic agents using Azure AI Foundry Agent Service",
        "✅ Implemented custom tools and function calling",
        "✅ Built multi-step reasoning capabilities",
        "✅ Managed conversation state and memory",
        "✅ Handled tool execution and error scenarios",
        "✅ Analyzed agent performance and behavior",
        "✅ Learned production best practices"
    ]
    
    for achievement in achievements:
        print(achievement)
    
    print(f"\n🔧 Technical Skills Gained:")
    skills = [
        "• Azure AI Foundry Agent Service usage",
        "• Custom tool creation and implementation",
        "• Function calling and tool execution handling",
        "• Multi-step reasoning and planning",
        "• Conversation thread management",
        "• Agent monitoring and performance analysis",
        "• Production deployment considerations"
    ]
    
    for skill in skills:
        print(skill)
    
    print(f"\n🚀 Next Workshop Preview:")
    print("Workshop 4: Evaluations")
    print("• Evaluate agent performance and quality")
    print("• Implement safety and reliability testing")
    print("• Create custom evaluation metrics")
    print("• Automated testing frameworks for AI")
    
    # Cleanup (optional - comment out if you want to keep agents for later use)
    cleanup_choice = input(f"\n🧹 Clean up created agents? (y/n): ").lower().strip()
    
    if cleanup_choice == 'y':
        print("🧹 Cleaning up agents...")
        agents_to_cleanup = [
            ("Basic Agent", basic_agent),
            ("Tool Agent", tool_agent), 
            ("Planning Agent", planning_agent)
        ]
        
        for name, agent in agents_to_cleanup:
            if agent:
                try:
                    # Note: Agent deletion might not be immediately available in all regions
                    # agents_client.delete_agent(agent.id)
                    print(f"📝 Note: {name} ({agent.id}) - Manual cleanup recommended")
                except Exception as e:
                    print(f"⚠️ Could not delete {name}: {e}")
    else:
        print("📝 Agents preserved for further experimentation")
        print("🆔 Agent IDs for reference:")
        if basic_agent:
            print(f"   Basic Agent: {basic_agent.id}")
        if tool_agent:
            print(f"   Tool Agent: {tool_agent.id}")
        if planning_agent:
            print(f"   Planning Agent: {planning_agent.id}")
    
    print(f"\n💡 Homework:")
    homework = [
        "Experiment with different tool combinations",
        "Try creating agents with domain-specific tools",
        "Implement error handling improvements",
        "Monitor agent conversations in Azure AI Foundry portal",
        "Explore the Azure AI Agent Service documentation"
    ]
    
    for item in homework:
        print(f"• {item}")

workshop_summary_and_cleanup()
</VSCode.Cell>

<VSCode.Cell language="markdown">
## 🔧 Troubleshooting Guide

### Common Issues and Solutions

#### Agent Creation Fails
- **Problem**: "Agent service not available" or permission errors
- **Solution**: 
  - Verify Azure AI Foundry project has Agent Service enabled
  - Check your Azure permissions and role assignments
  - Ensure you're using a supported model deployment

#### Tool Calls Not Working
- **Problem**: Agent doesn't use tools or tool calls fail
- **Solution**:
  - Check tool function descriptions are clear and specific
  - Verify tool parameter schemas are correct
  - Implement proper error handling in tool functions
  - Check that tool call handling logic is working

#### Memory/Context Issues  
- **Problem**: Agent doesn't remember previous conversation
- **Solution**:
  - Ensure you're using the same thread ID across messages
  - Check thread message history with list_messages
  - Verify conversation context isn't being lost

#### Performance Issues
- **Problem**: Agent responses are slow or time out
- **Solution**:
  - Implement timeouts for tool calls
  - Optimize tool function execution time
  - Use simpler tool descriptions
  - Check Azure service quotas and limits

### 📚 Additional Resources

- [Azure AI Foundry Agent Service Documentation](https://learn.microsoft.com/azure/ai-foundry/how-to/develop/agents)
- [Azure AI Projects SDK Documentation](https://learn.microsoft.com/python/api/azure-ai-projects/)
- [OpenAI Function Calling Guide](https://platform.openai.com/docs/guides/function-calling)
- [Azure AI Foundry Portal](https://ai.azure.com/)

### 🎮 Try These Extensions

1. **Weather Alerts**: Create a tool that monitors weather changes
2. **Cost Optimizer**: Build a tool that suggests cost optimization strategies
3. **Multi-Agent Systems**: Create multiple agents that collaborate
4. **Custom Memory**: Implement persistent memory across sessions
5. **Approval Workflows**: Add human approval for sensitive tool calls
</VSCode.Cell>
```